# GRAPE Automation applied to EDDK

This notebook brings together the functionality of the `GRAPE_traffic` library with GRAPE. It does the following:

1. defines the airport of interest, as well as start and end times to create studies (as the traffic library loads everything into memory, we split the data per month. The raw data for a single month contains more than 2GB of data).
2. if processed data is already cached, it is loaded and steps 3. to 9. are skipped.
3. fetches the airport traffic data, based on the bounding box defined in the *GRAPE_traffic.conf* file.
4. cleans any invalid data (nan values, outliers, duplicates).
5. writes flight features to *flights unclean.csv*.
6. performs integrity checks on each flight (aligned with a runway, has an ANP aircraft, ...).
7. writes flight features to *flights clean.csv*.
8. computes thrust.
9. saves data to cache.
10. for aircraft discarded in the filtering process, tries to find substitutes.
11. creates a GRAPE study and inserts the data into it, adds scenarios and calculation runs to the study.
12. calls GRAPE.exe and executes all the runs.
13. reads the outputs in the GRAPE study and transforms it to *.csv* files containing noise results per flight/receptor and emissions results per flight.

In [ ]:
import sqlite3
from pathlib import Path

import pandas as pd

from GRAPE_traffic import (
    AirportOpensky,
    AirportTraffic,
    GrapeTraffic,
)
from GRAPE_traffic.GrapeTraffic import (
    emissions_id,
    emissions_id_lto_cycle,
    noise_id,
    performance_id,
    scenario_id,
)

start_times = [
    "2019-01-01 00:00:00",
    "2019-02-01 00:00:00",
    "2019-03-01 00:00:00",
    "2019-04-01 00:00:00",
    "2019-05-01 00:00:00",
    "2019-06-01 00:00:00",
    "2019-07-01 00:00:00",
    "2019-08-01 00:00:00",
    "2019-09-01 00:00:00",
    "2019-10-01 00:00:00",
    "2019-11-01 00:00:00",
    "2019-12-01 00:00:00",
]

stop_times = [
    "2019-01-31 23:59:59",
    "2019-02-28 23:59:59",
    "2019-03-31 23:59:59",
    "2019-04-30 23:59:59",
    "2019-05-31 23:59:59",
    "2019-06-30 23:59:59",
    "2019-07-31 23:59:59",
    "2019-08-31 23:59:59",
    "2019-09-30 23:59:59",
    "2019-10-31 23:59:59",
    "2019-11-30 23:59:59",
    "2019-12-31 23:59:59",
]

airport = "EDDK"

output_folder = Path("out/eddk")

path_flights_unclean = output_folder / "flights unclean.csv"
path_flights_clean = output_folder / "flights clean.csv"
path_flights_substitutes = output_folder / "flights substitutes.csv"
path_flights_emissions = output_folder / "flights emissions.csv"
path_noise_events = output_folder / "noise events.csv"
path_noise_thresholds = "data/EDDK 2019/Noise Thresholds.xlsx"

fill_invalid_flag = True

In [ ]:
def get_airport_traffic(airport: str, start_time: str, stop_time: str) -> AirportTraffic:
    path = output_folder / f"{airport}_{start_time}_{stop_time}"

    print(f"Getting data for {airport} from {start_time} to {stop_time}")

    if path.exists():
        return AirportTraffic.from_folder(path)

    apt = AirportOpensky(airport, start_time, stop_time)
    arr = apt.fetch_arrivals()
    dep = apt.fetch_departures()

    print("Cleaning data...")
    apt_traffic = (
        AirportTraffic(arr, dep)
        .clean()
        .assign_ids(id_postfix=f"{pd.to_datetime(start_time).month:02d}")
        .cumulative_distance()
    )
    f = apt_traffic.list_all()

    if f is not None:
        header_unclean = not path_flights_unclean.exists()

        f.to_csv(path_flights_unclean, index=False, header=header_unclean, mode="a")

    apt_traffic = apt_traffic.clean_arrivals().clean_departures()
    f = apt_traffic.list_all()

    if f is not None:
        header_clean = not path_flights_clean.exists()

        f.to_csv(path_flights_clean, index=False, header=header_clean, mode="a")

    print("Computing Thrust...")
    apt_traffic.compute_thrust()

    apt_traffic.save(path)
    return apt_traffic

In [ ]:
def fill_invalid(month: int) -> pd.Series:
    print("Filling invalid flights...")

    def get_od(row):
        return row["origin"] if row["operation"] == "Arrival" else row["destination"]

    all = pd.read_csv(path_flights_unclean)
    all["month"] = all["flight_id"].str.extract(r"\w{3}(\d+)_.*", expand=False).astype(int)
    all = all.loc[all["month"] == month]
    all["time"] = pd.to_datetime(all["time"], utc=True)
    all["od"] = all.apply(get_od, axis="columns")

    valid = pd.read_csv(path_flights_clean)
    valid["month"] = valid["flight_id"].str.extract(r"\w{3}(\d+)_.*", expand=False).astype(int)
    valid = valid.loc[valid["month"] == month]
    valid["time"] = pd.to_datetime(valid["time"], utc=True)
    valid["od"] = valid.apply(get_od, axis="columns")
    valid = valid.drop(columns=["origin", "destination", "go_around", "changed_runway"])

    invalid = all.loc[~all["flight_id"].isin(valid["flight_id"])]

    # Remove flights which can't be filled and get od
    invalid = invalid.dropna(subset=["runway", "icao24", "aircraft_id"])
    invalid = invalid.loc[invalid["anp_power_param"].isin(["CNT (lb)", r"CNT (% of Max Static Thrust)"])]
    invalid = invalid.drop(
        columns=[
            "origin",
            "destination",
            "go_around",
            "changed_runway",
            "anp",
            "anp_power_param",
            "month",
        ]
    )

    def get_closest_flight(df):
        return df.loc[
            (df["time"] - df["time_new"]).abs().idxmin(),
            ["flight_id", "flight_id_new"],
        ]

    rules = [
        ["operation", "runway", "icao24", "od"],
        ["operation", "runway", "aircraft_id", "od"],
        ["operation", "runway", "icao24"],
        ["operation", "runway", "aircraft_id"],
    ]

    substitutes = []

    for i, rule in enumerate(rules):
        df = (
            invalid.merge(valid.dropna(subset=rule), on=rule, suffixes=("", "_new"))
            .groupby("flight_id")
            .apply(get_closest_flight)
        )
        df["rule"] = i

        substitutes.append(df)

    substitutes = pd.concat(substitutes, ignore_index=True).drop_duplicates(subset="flight_id")
    header_substitutes = not path_flights_substitutes.exists()
    substitutes.to_csv(
        path_flights_substitutes,
        header=header_substitutes,
        mode="a",
        index=False,
    )

    counts = substitutes["flight_id_new"].value_counts()
    return counts

In [ ]:
for start_time_str, stop_time_str in zip(start_times, stop_times):
    start_time = pd.to_datetime(start_time_str).tz_localize("UTC")
    stop_time = pd.to_datetime(stop_time_str).tz_localize("UTC")
    start_time_str = start_time.strftime("%Y-%m-%d")
    stop_time_str = stop_time.strftime("%Y-%m-%d")

    # Get airport traffic
    apt_traffic = get_airport_traffic(airport, start_time_str, stop_time_str)

    # Get operation count to fill invalid flights
    if fill_invalid_flag:
        op_counts = fill_invalid(start_time.month)

    # Generate and run GRAPE study
    print("Generating GRAPE file...")
    default_name = f"{airport}_{start_time_str}_{stop_time_str}"
    path_study = (output_folder / default_name / default_name).with_suffix(".grp")
    grape_traffic = GrapeTraffic(apt_traffic, "data/EDDK 2019/receptors.csv", study_path=path_study)
    grape_traffic.generate_tables()
    ops = grape_traffic.grape_tables["operations_tracks_4d"]
    if fill_invalid_flag:
        ops = ops.merge(
            op_counts,
            how="left",
            left_on="id",
            right_index=True,
            suffixes=("", "_new"),
        )
        ops["count_new"] = ops["count_new"].fillna(0)
        ops["count"] += ops["count_new"]
        grape_traffic.grape_tables["operations_tracks_4d"] = ops.drop(columns=["count_new"])
    grape_traffic.generate_grape()

    # Run GRAPE
    print("Running study...")
    grape_traffic.run()

## Emissions output
 
 Creates a *flights emissions.csv* file with columns:
 - `flight_id`
 - `duration`
 - `fuel`
 - `hc`
 - `co`
 - `nox`
 - `nvpm`
 - `nvpm_number`
 - `fuel_lto_cycle`
 - `hc_lto_cycle`
 - `co_lto_cycle`
 - `nox_lto_cycle`
 - `nvpm_lto_cycle`
 - `nvpm_number_lto_cycle`

In [ ]:
for start_time_str, stop_time_str in zip(start_times, stop_times):
    start_time = pd.to_datetime(start_time_str).tz_localize("UTC")
    stop_time = pd.to_datetime(stop_time_str).tz_localize("UTC")
    start_time_str = start_time.strftime("%Y-%m-%d")
    stop_time_str = stop_time.strftime("%Y-%m-%d")
    default_name = f"{airport}_{start_time_str}_{stop_time_str}"
    path_study = (output_folder / default_name / default_name).with_suffix(".grp")

    print(f"Processing flights from {path_study}.")
    # Connections
    con = sqlite3.connect(path_study)

    # Read tables
    emi_lto_cycle = pd.read_sql(
        f"SELECT operation_id, operation, segment_number, fuel, hc, co, nox, nvpm, nvpm_number FROM emissions_run_output_segments WHERE emissions_run_id = '{emissions_id_lto_cycle}'",
        con=con,
    )
    emi_segs = pd.read_sql(
        f"SELECT operation_id, operation, segment_number, fuel, hc, co, nox, nvpm, nvpm_number FROM emissions_run_output_segments WHERE emissions_run_id = '{emissions_id}'",
        con=con,
    )
    perf_pts = pd.read_sql(
        f"SELECT operation_id, point_number, time FROM performance_run_output_points WHERE scenario_id = '{scenario_id}'",
        con=con,
    )

    # LTO Cycle
    vars = ["fuel", "hc", "co", "nox", "nvpm", "nvpm_number"]

    emi_lto_cycle.loc[
        (emi_lto_cycle["operation"] == "Arrival") & emi_lto_cycle["segment_number"].isin([2, 3]),
        vars,
    ] = 0
    emi_lto_cycle.loc[
        (emi_lto_cycle["operation"] == "Departure") & (emi_lto_cycle["segment_number"] == 1),
        vars,
    ] = 0
    emi_lto_cycle = emi_lto_cycle.drop(columns="segment_number")
    agg = {}
    agg["operation"] = "first"
    for var in vars:
        agg[var] = "sum"
    emi_lto_cycle = emi_lto_cycle.groupby("operation_id").agg(agg)

    # Segments
    emi_segs_times = emi_segs[["operation_id", "segment_number"]].merge(
        perf_pts,
        left_on=["operation_id", "segment_number"],
        right_on=["operation_id", "point_number"],
        how="left",
    )
    emi_segs_times["time"] = pd.to_datetime(emi_segs_times["time"], utc=True)

    emi_segs = emi_segs.drop(columns="segment_number").groupby("operation_id").agg(agg)
    emi_segs_times = emi_segs_times.groupby("operation_id")["time"].agg(lambda x: (x.max() - x.min()).total_seconds())

    emi_segs = (
        emi_segs.merge(emi_segs_times, on="operation_id")
        .drop(columns=["operation"])
        .rename(columns={"time": "duration"})[
            [
                "duration",
                "fuel",
                "hc",
                "co",
                "nox",
                "nvpm",
                "nvpm_number",
            ]
        ]
    )

    # Merge segments and LTO Cycle
    emi = emi_segs.merge(
        emi_lto_cycle.drop(columns="operation"),
        on="operation_id",
        suffixes=("", "_lto_cycle"),
    )
    emi = emi.rename(columns={"operation_id": "flight_id"})

    # Write to .csv
    header = not path_flights_emissions.exists()
    emi.to_csv(path_flights_emissions, header=header, mode="a")

    # Close connections
    con.close()

## Noise Output

 Creates a *flights noise.csv* file with columns:
 - `flight_id`
 - `receptor_id`
 - `count`
 - `lamax`
 - `sel`
 - `energy`

 Noise events below the thresholds defined by the airport will be discarded.

In [ ]:
for start_time_str, stop_time_str in zip(start_times, stop_times):
    start_time = pd.to_datetime(start_time_str).tz_localize("UTC")
    stop_time = pd.to_datetime(stop_time_str).tz_localize("UTC")
    start_time_str = start_time.strftime("%Y-%m-%d")
    stop_time_str = stop_time.strftime("%Y-%m-%d")
    default_name = f"{airport}_{start_time_str}_{stop_time_str}"
    path_study = (output_folder / default_name / default_name).with_suffix(".grp")

    # Connections
    con = sqlite3.connect(path_study)

    ns = pd.read_sql(
        f"SELECT t1.operation_id, t1.receptor_id, t1.maximum_db, t1.exposure_db, t2.time, t2.count "
        f"FROM noise_run_output_single_event t1 JOIN operations_tracks_4d t2 ON t1.operation_id = t2.id "
        f"WHERE "
        f"t1.scenario_id = '{scenario_id}' AND "
        f"t1.performance_run_id = '{performance_id}' AND "
        f"t1.noise_run_id = '{noise_id}'",
        con=con,
    )
    ns["time"] = pd.to_datetime(ns["time"], utc=True)

    # Delete events based on thresholds
    thresholds = pd.read_excel(path_noise_thresholds, usecols="A:B")
    ns = pd.merge(ns, thresholds, how="left", on="receptor_id")
    ns = ns.loc[~((ns["constant"] == "yes") & (ns["maximum_db"] < 65.0))]
    day = (ns["time"].dt.hour >= 6) & (ns["time"].dt.hour <= 22)
    ns = ns.loc[~((ns["constant"] == "no") & day & (ns["maximum_db"] < 65.0))]
    ns = ns.loc[~((ns["constant"] == "no") & ~day & (ns["maximum_db"] < 63.0))]

    # Organize
    ns = ns.drop(columns=["time", "constant"]).rename(
        columns={
            "operation_id": "flight_id",
            "maximum_db": "lamax",
            "exposure_db": "sel",
        }
    )[["flight_id", "receptor_id", "count", "lamax", "sel"]]

    # Write to .csv
    header = not path_noise_events.exists()
    ns.to_csv(path_noise_events, header=header, mode="a", index=False)

    # Close connections
    con.close()

## Emissions Analysis

In [ ]:
d = pd.read_csv(path_flights_emissions)
d = d.merge(
    pd.read_csv(path_flights_clean, usecols=["flight_id", "operation", "aircraft_id"]),
    left_on="operation_id",
    right_on="flight_id",
    how="left",
)
d = d.loc[d["aircraft_id"].isin(d["aircraft_id"].value_counts().head(5).index)]
d["hc"] *= 1000  # kg2g
d["duration"] /= 60  # sec 2 min
d["hc_lto_cycle"] *= 1000  # kg2g
d["co"] *= 1000  # kg2g
d["co_lto_cycle"] *= 1000  # kg2g
d["nox"] *= 1000  # kg2g
d["nox_lto_cycle"] *= 1000  # kg2g
d["nvpm"] *= 1e6  # kg2mg
d["nvpm_lto_cycle"] *= 1e6  # kg2mg

arr = d.loc[d["operation"] == "Arrival"]
dep = d.loc[d["operation"] == "Departure"]

In [ ]:
arr.groupby("aircraft_id").mean(numeric_only=True).to_csv(
    output_folder / "flights emissions top10 arrival.csv", float_format="%.2f"
)
dep.groupby("aircraft_id").mean(numeric_only=True).to_csv(
    output_folder / "out/flights emissions top10 departure.csv",
    float_format="%.2f",
)

In [ ]:
arr.groupby("aircraft_id").std(numeric_only=True).round(1).to_csv(
    output_folder / "flights emissions top5 arrival std rounded.csv"
)
dep.groupby("aircraft_id").std(numeric_only=True).round(1).to_csv(
    output_folder / "flights emissions top5 departure std rounded.csv"
)

In [ ]:
t = arr.sum(numeric_only=True)
print(f"Arrival duration diff: {t['duration'] - 4.0 * len(arr)} min")
print(f"Arrival fuel diff: {t['fuel'] - t['fuel_lto_cycle']} kg")

t = dep.sum(numeric_only=True)
print(f"Departure duration diff: {t['duration'] - 2.9 * len(dep)} min")
print(f"Departure fuel diff: {t['fuel'] - t['fuel_lto_cycle']} kg")